In [ ]:
!pip install torch torchvision numpy opencv-python grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s e

In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms

# Load the high-resolution .tiff image
image_path = '/content/aoi0.tif'
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

# Convert the img to RGB if the image has more than one channel
if len(image.shape) == 3 and image.shape[2] == 4:
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
elif len(image.shape) == 2 or image.shape[2] == 1:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# Resize the image to match the model's expected input size (e.g., 256x256)
resize = transforms.Resize((256, 256))
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

# Apply transformations
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_pil = transforms.ToPILImage()(image)
image_resized = resize(image_pil)
image_tensor = to_tensor(image_resized)
image_tensor = normalize(image_tensor).unsqueeze(0)


In [ ]:
from dehazeformer import DehazeFormer

# Initialize the model
model = DehazeFormer()
model.load_state_dict(torch.load('/content/dehazeformer-b.pth'))
model.eval()

In [ ]:
with torch.no_grad():
    dehazed_image_tensor = model(image_tensor)

# Post-process the output to convert it back to an image
dehazed_image = dehazed_image_tensor.squeeze(0).cpu().numpy()
dehazed_image = np.transpose(dehazed_image, (1, 2, 0))  # Change from (C, H, W) to (H, W, C)
dehazed_image = (dehazed_image * 0.5 + 0.5) * 255  # Denormalize
dehazed_image = dehazed_image.astype(np.uint8)

# Save or display the dehazed image
cv2.imwrite('dehazed_image.tiff', cv2.cvtColor(dehazed_image, cv2.COLOR_RGB2BGR))


In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# Define a reshape_transform function for the transformer model
def reshape_transform(tensor, height=16, width=16):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))
    result = result.permute(0, 3, 1, 2)
    return result

# Select the target layer for Grad-CAM
target_layer = model.encoder.layers[-1].self_attn

# Initialize Grad-CAM
cam = GradCAM(model=model, target_layers=[target_layer], reshape_transform=reshape_transform)

# Generate the CAM
grayscale_cam = cam(input_tensor=image_tensor, targets=None)

# Visualize the CAM
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(np.array(image_resized) / 255., grayscale_cam, use_rgb=True)

# Save or display the visualization
cv2.imwrite('grad_cam_visualization.tiff', cv2.cvtColor(visualization, cv2.COLOR_RGB2BGR))
